In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import model
import datasets

from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# Make sure to use the GPU. The following line is just a check to see if GPU is availables
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
# load your dataset and dataloader
# feel free to change header of bird_dataset class
# root = 'path to images/'
root = 'birds_dataset/'
train_dataset = datasets.bird_dataset(root,'train_list.txt')
test_dataset = datasets.bird_dataset(root,'test_list.txt')

# Fill in optional arguments to the dataloader as you need it
# batch_size = 256

train_dataloader = DataLoader(train_dataset, batch_size=4,
                                          shuffle=True, num_workers=2)
test_dataloader = DataLoader(test_dataset, batch_size=4,
                                         shuffle=False, num_workers=2)

In [4]:
#create validation set?
# train_ds, valid_ds = torch.utils.data.random_split(train_dataloader.dataset, (len(train_dataset)*0.75, len(train_dataset)*0.25))
# # print(len(train_dataloader.dataset))

In [5]:
for i in range(len(train_dataset)):
    sample = train_dataset[i]

    print(i, sample['image'].shape, sample['classification'])

    if i == 3:
        break
# print(train_dataset[1])

TypeError: tuple indices must be integers or slices, not str

In [6]:
# Create NN model object
nn_model = model.baseline_Net(classes = 20)
nn_model.to(device)


baseline_Net(
  (b1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (b2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (b3): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (b4): Sequential(
    (0): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
    (1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))
    (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU(inplace=True)
  )
  (avg_pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc1): Sequential(
    (0): Lin

In [7]:
# Create loss functions, optimizers
# For baseline model use this
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(nn_model.parameters(), lr=1e-4)
# Initialize weights

In [8]:
print(train_dataloader.dataset[1])

(tensor([[[ 1.1015,  1.1016,  1.1275,  ...,  1.1178,  1.1314,  1.1248],
         [ 1.1076,  1.1192,  1.1096,  ...,  1.1025,  1.1155,  1.1518],
         [ 1.1187,  1.1417,  1.1428,  ...,  1.1136,  1.1350,  1.1187],
         ...,
         [-0.5627, -0.5894, -0.5433,  ...,  0.6847,  0.6797,  0.6508],
         [-0.5203, -0.5646, -0.5397,  ...,  0.8398,  0.8082,  0.8082],
         [-0.3421, -0.3603, -0.3037,  ...,  0.9378,  0.9202,  0.9202]],

        [[ 1.2206,  1.2206,  1.2471,  ...,  1.2197,  1.2334,  1.2227],
         [ 1.2082,  1.2200,  1.2102,  ...,  1.2041,  1.2163,  1.2319],
         [ 1.1856,  1.2091,  1.2102,  ...,  1.2154,  1.2373,  1.2206],
         ...,
         [-0.1689, -0.2091, -0.1919,  ...,  0.7261,  0.7418,  0.7319],
         [-0.1275, -0.1452, -0.1773,  ...,  0.8871,  0.9032,  0.9032],
         [-0.0194, -0.0047, -0.0239,  ...,  1.0300,  1.0177,  1.0177]],

        [[ 1.5245,  1.5245,  1.5510,  ...,  1.5585,  1.5721,  1.5594],
         [ 1.5183,  1.5301,  1.5204,  ...,  

In [10]:
# train your model
# For each epoch iterate over your dataloaders/datasets, pass it to your NN model, get output, calculate loss and
# backpropagate using optimizer
num_epochs = 25
loss = []
running_loss = 0.0
# for iteration in range(num_epochs):
#     nn_model(train_dataloader)
for epoch in range(1):
    for i, data in enumerate(train_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = nn_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 4 == 3:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 4))
                running_loss = 0.0

    print('Finished Training')

[1,     4] loss: 0.006
[1,     8] loss: 0.006
[1,    12] loss: 0.006
[1,    16] loss: 0.006
[1,    20] loss: 0.006
[1,    24] loss: 0.006
[1,    28] loss: 0.006
[1,    32] loss: 0.006
[1,    36] loss: 0.006
[1,    40] loss: 0.006
[1,    44] loss: 0.006
[1,    48] loss: 0.006
[1,    52] loss: 0.006
[1,    56] loss: 0.006
[1,    60] loss: 0.006
[1,    64] loss: 0.006
[1,    68] loss: 0.006
[1,    72] loss: 0.006
[1,    76] loss: 0.006
[1,    80] loss: 0.006
[1,    84] loss: 0.006
[1,    88] loss: 0.006
[1,    92] loss: 0.006
[1,    96] loss: 0.006
[1,   100] loss: 0.005
[1,   104] loss: 0.006
[1,   108] loss: 0.006
[1,   112] loss: 0.006
[1,   116] loss: 0.006
[1,   120] loss: 0.006
[1,   124] loss: 0.006
[1,   128] loss: 0.005
[1,   132] loss: 0.006
[1,   136] loss: 0.006
[1,   140] loss: 0.006
[1,   144] loss: 0.006
[1,   148] loss: 0.006
Finished Training


In [ ]:
# Save your model/best model